### convert PDF to  IMG and  OCR To extract Text

In [4]:
import pandas as pd
from pdf2image import convert_from_path
import os
from paddleocr import PaddleOCR
import numpy as np

In [2]:
import paddleocr

In [3]:
ocr = PaddleOCR(use_angle_cls=True, lang="en") 

[2025/03/31 11:43:44] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Shripad/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Shripad/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 3

In [4]:
images = convert_from_path("./test2.pdf")

In [5]:
len(images)

31

In [27]:
text = ""
for i, img in enumerate(images):
    img = np.array(img)
    result = ocr.ocr(img, cls=True) 
    for line in result:
        for word_info in line:
            text += word_info[1][0] + " "  
    text += "\n\n" 

[2025/03/31 12:03:22] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.4393618106842041
[2025/03/31 12:03:23] ppocr DEBUG: cls num  : 1, elapsed : 0.27399420738220215
[2025/03/31 12:03:23] ppocr DEBUG: rec_res num  : 1, elapsed : 0.4400196075439453
[2025/03/31 12:03:24] ppocr DEBUG: dt_boxes num : 32, elapsed : 0.3721754550933838
[2025/03/31 12:03:24] ppocr DEBUG: cls num  : 32, elapsed : 0.2492833137512207
[2025/03/31 12:03:45] ppocr DEBUG: rec_res num  : 32, elapsed : 21.264811992645264
[2025/03/31 12:03:46] ppocr DEBUG: dt_boxes num : 48, elapsed : 0.6108613014221191
[2025/03/31 12:03:46] ppocr DEBUG: cls num  : 48, elapsed : 0.2969691753387451
[2025/03/31 12:03:55] ppocr DEBUG: rec_res num  : 48, elapsed : 8.625164985656738
[2025/03/31 12:03:57] ppocr DEBUG: dt_boxes num : 46, elapsed : 1.0719332695007324
[2025/03/31 12:03:57] ppocr DEBUG: cls num  : 46, elapsed : 0.47006750106811523
[2025/03/31 12:04:07] ppocr DEBUG: rec_res num  : 46, elapsed : 10.22400188446045
[2025/03/31 12:04:08] p

In [28]:
text

'This page intentionally left blank \n\nCHAPTER 4 Cloud Computing Architecture The term cloud computing is a wide umbrella encompassing many different things; lately it has become a buzzword that is easily misused to revamp existing technologies and ideas for the public What makes cloud computing so interesting to IT stakeholders and research practitioners? How does it introduce innovation into the field of distributed computing? This chapter addresses all these questions and characterizes the phenomenon. It provides a reference model that serves as a basis for discussion of cloud computing technologies. 4.1Introduction Utility-oriented data centers are the first outcome of cloud computing, and they serve as the infra structure through which the services are implemented and delivered. Any cloud service, whether virtual hardware, development platform, or application software, relies on a distributed infrastruc ture owned by the provider or rented from a third party. As noted in the prev

In [29]:
with open("extracted_text.txt", "w", encoding="utf-8") as f:
    f.write(text)

In [ ]:
text

In [89]:
def clean_text(text2):
    text2 = re.sub(r"\d+\s*(CHAPTER\s*\d+)?", "", text2)  # Remove numbers & chapter references
    text2 = re.sub(r"-\s+", "", text2)  # Fix broken words
    return text2.strip()

cleaned_text = clean_text(text)

In [90]:
cleaned_text

'This page intentionally left blank \n\nCHAPTER Cloud Computing Architecture The term cloud computing is a wide umbrella encompassing many different things; lately it has become a buzzword that is easily misused to revamp existing technologies and ideas for the public What makes cloud computing so interesting to IT stakeholders and research practitioners? How does it introduce innovation into the field of distributed computing? This chapter addresses all these questions and characterizes the phenomenon. It provides a reference model that serves as a basis for discussion of cloud computing technologies. .Introduction Utility-oriented data centers are the first outcome of cloud computing, and they serve as the infra structure through which the services are implemented and delivered. Any cloud service, whether virtual hardware, development platform, or application software, relies on a distributed infrastruc ture owned by the provider or rented from a third party. As noted in the previous

### Creating Question Answer using Gemini

In [5]:
with open("new_extracted_text.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [7]:
def generate_qa(text):
    prompt = f"""Generate 3 question-answer pairs based on the following text:
    {text}
    and the resultant structure should  be json structure 
    "question":
    "answer":
    """

    response = model.generate_content(prompt)
    return response.text if response else None

In [15]:
paragraphs = text.split("\n\n")
import google.generativeai as genai
from tqdm import tqdm
import time
import json
qa_list = []
genai.configure(api_key="AIzdpyA78u1MehLi5Ymfy30c9wZALUgZEkGC_u0")
model = genai.GenerativeModel("models/gemini-1.5-pro")
tries = 0
for paragraph in tqdm(paragraphs[25:30], desc="Processing Paragraphs"):
    qa_pair = generate_qa(paragraph)
    qa_list.append(qa_pair) 
    time.sleep(tries)
    tries = tries+15
    if tries == 120:
        tries = 10

Processing Paragraphs: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:56<00:00, 35.23s/it]


In [17]:
with open("qa_pairs6.json", "w", encoding="utf-8") as json_file:
    json.dump(qa_list, json_file, indent=4, ensure_ascii=False)

In [30]:
with open("qa_pairs.json", "r") as file:
    json_data = json.load(file)

In [31]:
import re
qa_list = []

for response in json_data:
    clean_response = re.sub(r"```json|\n```", "", response).strip()
    
    try:
        data = json.loads(clean_response)
        if isinstance(data, list):
            for entry in data:
                if isinstance(entry, dict) and "question" in entry and "answer" in entry:
                    qa_list.append(entry)
    
    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        print(f"Faulty JSON: {clean_response}")

output_file = "new_qa_pairs.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(qa_list, f, indent=4, ensure_ascii=False)

print(f"JSON file saved as {output_file}")

JSON file saved as new_qa_pairs.json
